In [2]:
# Inverse Design of Patch Antennas - Polished Workload-Ready Version
# ======================================================

# 1. Environment Setup
# ---------------------
!pip install -q tensorflow numpy pandas matplotlib scikit-learn keras-tuner gradio

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
import gradio as gr
import json
import os

# Set random seeds for full reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# 2. Dataset Generation (Physics-Constrained)
# --------------------------------------------
c = 3e8  # Speed of light (m/s)
N = 30000  # Number of antenna samples

# Design parameters
f = np.random.uniform(1e9, 12e9, size=N)
er = np.random.uniform(2.2, 12.0, size=N)
h = np.random.uniform(0.5e-3, 3e-3, size=N)
loss_tangent = np.random.uniform(0.0001, 0.02, size=N)

# Analytical formulas for patch antenna
W = np.clip(c / (2*f) * np.sqrt(2/(er+1)), 1e-3, 300e-3)
eps_eff = (er+1)/2 + (er-1)/(2*np.sqrt(1+12*h/W))
delta_L = 0.412*h*((eps_eff+0.3)*(W/h+0.264))/((eps_eff-0.258)*(W/h+0.8))
L = np.clip(c / (2*f*np.sqrt(eps_eff)) - 2*delta_L, 1e-3, 300e-3)

# Add realistic fabrication noise
W_noisy = W * np.random.normal(1.0, 0.015, size=N)
L_noisy = L * np.random.normal(1.0, 0.015, size=N)

# Build dataset with physical enforcement: W >= 1.05*L
data = pd.DataFrame({
    'f_GHz': f/1e9,
    'W_mm': np.maximum(W_noisy*1e3, L_noisy*1e3 * 1.05),
    'L_mm': L_noisy*1e3,
    'er': er,
    'h_mm': h*1e3,
    'loss_tangent': loss_tangent
})

# 3. Data Preparation
# --------------------
X = data[['f_GHz', 'er']].values
y = data[['L_mm', 'W_mm']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# 4. Model Definition (Physics-Aware Loss)
# ----------------------------------------
def custom_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    constraint_penalty = tf.reduce_mean(tf.square(tf.maximum(1.05*y_pred[:,0] - y_pred[:,1], 0)))
    return mse + 0.001 * constraint_penalty

def constraint_satisfaction_metric(y_true, y_pred):
    return tf.reduce_mean(tf.cast(y_pred[:,1] >= 1.05 * y_pred[:,0], tf.float32)) * 100

def build_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            units=hp.Int('units1', 128, 512, step=64), activation='relu', input_shape=(2,)),
        tf.keras.layers.Dropout(hp.Float('dropout', 0.1, 0.5)),
        tf.keras.layers.Dense(
            units=hp.Int('units2', 64, 256, step=64), activation='relu'),
        tf.keras.layers.Dense(2)
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 5e-4, 1e-4])),
        loss=custom_loss,
        metrics=['mae', constraint_satisfaction_metric]
    )
    return model

# 5. Hyperparameter Tuning (Keras Tuner Polished)
# ------------------------------------------------
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_constraint_satisfaction_metric", direction="max"),
    max_trials=15,
    executions_per_trial=1,
    overwrite=True,
    directory='tuner_results',
    project_name='patch_antenna_inverse_design'
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_constraint_satisfaction_metric', patience=5, mode='max', restore_best_weights=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Start tuning
print("Starting hyperparameter tuning...")
tuner.search(X_train_scaled, y_train,
             epochs=60,
             batch_size=128,
             validation_split=0.2,
             callbacks=[early_stop, reduce_lr],
             verbose=2)

# 6. Best Model Training (Fine-tuned)
# -----------------------------------
best_hp = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hp)

# Full training on best model
print("Retraining best model...")
history = best_model.fit(
    X_train_scaled, y_train,
    epochs=150,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=2
)

# 7. Evaluation
# --------------
y_pred_scaled = best_model.predict(X_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

# Enforce constraint at prediction
y_pred[:,1] = np.maximum(y_pred[:,1], y_pred[:,0]*1.05)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
compliance = constraint_satisfaction_metric(tf.convert_to_tensor(y_test, dtype=tf.float32), tf.convert_to_tensor(y_pred, dtype=tf.float32)).numpy()

print("\n=== Evaluation Results ===")
print(f"MAE: {mae:.3f} mm")
print(f"R² Score: {r2:.4f}")
print(f"Physics Compliance: {compliance:.2f}%")

# 8. Visualization
# -----------------
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history['constraint_satisfaction_metric'], label='Train Compliance')
plt.plot(history.history['val_constraint_satisfaction_metric'], label='Val Compliance')
plt.title('Physics Constraint Satisfaction (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("training_curves.png", dpi=300)
plt.close()

# 9. Saving Model and Hyperparameters
# ------------------------------------
if not os.path.exists('outputs'):
    os.makedirs('outputs')

best_model.save("outputs/final_patch_antenna_model.keras")

with open("outputs/best_hyperparameters.json", "w") as f:
    json.dump({
        'units1': best_hp.get('units1'),
        'units2': best_hp.get('units2'),
        'dropout': best_hp.get('dropout'),
        'learning_rate': best_hp.get('learning_rate')
    }, f, indent=4)

# 10. Gradio Interface (After training ends)
# -------------------------------------------
def plot_antenna(L, W):
    fig, ax = plt.subplots(figsize=(6,4))
    ax.add_patch(plt.Rectangle((0,0), L, W, fill=False, edgecolor='blue', linewidth=2))
    ax.set_xlim(0, max(L,W)+10)
    ax.set_ylim(0, max(L,W)+10)
    ax.set_aspect('equal')
    ax.set_title(f"Patch Antenna\nL: {L:.1f} mm | W: {W:.1f} mm")
    ax.grid(True)
    return fig

def predict_patch(f_GHz, er):
    X_input = X_scaler.transform([[f_GHz, er]])
    pred = best_model.predict(X_input, verbose=0)
    L, W = y_scaler.inverse_transform(pred)[0]
    W = max(W, 1.05*L)
    text_output = f"Predicted L = {L:.2f} mm, W = {W:.2f} mm"
    return text_output, plot_antenna(L, W)

iface = gr.Interface(
    fn=predict_patch,
    inputs=[
        gr.Slider(1.0, 12.0, value=2.4, step=0.1, label="Frequency (GHz)"),
        gr.Slider(2.2, 12.0, value=4.4, step=0.1, label="Dielectric Constant (εᵣ)")
    ],
    outputs=[
        gr.Textbox(label="Predicted Dimensions"),
        gr.Plot(label="Patch Antenna Geometry")
    ],
    title="Patch Antenna Designer",
    description="Deep Learning-based Inverse Design | Physics-Constrained | Heavy Dataset"
)

iface.launch(share=True)


Trial 15 Complete [00h 00m 11s]
val_constraint_satisfaction_metric: 100.0

Best val_constraint_satisfaction_metric So Far: 100.0
Total elapsed time: 00h 02m 58s
Retraining best model...
Epoch 1/150
150/150 - 3s - 19ms/step - constraint_satisfaction_metric: 99.9740 - loss: 189.1965 - mae: 8.4241 - val_constraint_satisfaction_metric: 100.0000 - val_loss: 35.4863 - val_mae: 3.9178 - learning_rate: 5.0000e-04
Epoch 2/150
150/150 - 1s - 6ms/step - constraint_satisfaction_metric: 100.0000 - loss: 30.8969 - mae: 3.4402 - val_constraint_satisfaction_metric: 100.0000 - val_loss: 26.8794 - val_mae: 3.2600 - learning_rate: 5.0000e-04
Epoch 3/150
150/150 - 1s - 7ms/step - constraint_satisfaction_metric: 100.0000 - loss: 24.6879 - mae: 3.0029 - val_constraint_satisfaction_metric: 100.0000 - val_loss: 21.9152 - val_mae: 2.8568 - learning_rate: 5.0000e-04
Epoch 4/150
150/150 - 1s - 7ms/step - constraint_satisfaction_metric: 100.0000 - loss: 19.8984 - mae: 2.5540 - val_constraint_satisfaction_metric: 